# Test Database with SQLAlchemy

## Define Service Class for Database Queries using SQLAlchemy

In [7]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import or_
from db.movie_app_db import Movie, MovieGenre, Genre, MovieKeyword, Keyword,\
   MovieCredit, Person, MovieStudio, Studio
from sqlalchemy import create_engine
import pandas as pd

database_path = 'sqlite:///movie_app.db'

class MovieService:
    def __init__(self):
        # Connect to the SQLite database
        engine = create_engine(database_path)
        Session = sessionmaker(bind=engine)
        self.session = Session()
    
    def to_dataframe(self, object_list: list) -> pd.DataFrame: 
          # Convert each object into a dictionary of its attributes
        data = [obj.__dict__ for obj in object_list]

        # Create a pandas DataFrame from the list of dictionaries
        return pd.DataFrame(data)

    # 1. Query by title
    def query_by_title(self, search_string: str, exact_match: bool = False):
        if exact_match:
            # Exact title match
            return self.to_dataframe(self.session.query(Movie).filter(Movie.movie_name == search_string).all())
        else:
            # Wildcard match (case insensitive search)
            search_pattern = f"%{search_string}%"
            return self.to_dataframe(self.session.query(Movie).filter(Movie.movie_name.ilike(search_pattern)).all())
            

    # 2. Query by genre
    def query_by_genre(self, genre_name: str)-> pd.DataFrame:
        # Join with Genre table through MovieGenre
        return self.to_dataframe((
            self.session.query(Movie)
            .join(MovieGenre)
            .join(Genre)
            .filter(Genre.genre_name == genre_name)
            .all()
        ))

    # 3. Query by release date
    def query_by_release_date(self, start_date=None, end_date=None):
        query = self.session.query(Movie)
        if start_date:
            query = query.filter(Movie.movie_release_date >= start_date)
        if end_date:
            query = query.filter(Movie.movie_release_date <= end_date)
        return query.all()

    # 4. Query by keyword
    def query_by_keyword(self, keyword_name: str):
        # Join with Keyword table through MovieKeyword
        return (
            self.session.query(Movie)
            .join(MovieKeyword)
            .join(Keyword)
            .filter(Keyword.keyword_name == keyword_name)
            .all()
        )

    # 5. Query by person (actor, director, etc.)
    def query_by_person(self, person_name: str):
        # Join with Person table through MovieCredit
        return (
            self.session.query(Movie)
            .join(MovieCredit)
            .join(Person)
            .filter(Person.name == person_name)
            .all()
        )

    # 6. Query by studio
    def query_by_studio(self, studio_name: str):
        # Join with Studio table through MovieStudio
        return (
            self.session.query(Movie)
            .join(MovieStudio)
            .join(Studio)
            .filter(Studio.studio_name == studio_name)
            .all()
        )

# Usage example:
# Assuming session is already created and connected to the database

# movie_service = MovieService(session)
# movies_by_title = movie_service.query_by_title("Star Wars", exact_match=False)
# movies_by_genre = movie_service.query_by_genre("Action")
# movies_by_release_date = movie_service.query_by_release_date("2000-01-01", "2020-01-01")
# movies_by_keyword = movie_service.query_by_keyword("Adventure")
# movies_by_person = movie_service.query_by_person("Harrison Ford")
# movies_by_studio = movie_service.query_by_studio("Warner Bros")


## Create Unittest for Search by Name

In [5]:
import unittest
import datetime



# given: a movie title

# when: we query the database for the movie title

# then: it should return the data for this movie

# Test case for the MovieService class
class TestMovieService(unittest.TestCase):
    def setUp(self):
        # Create a MovieService instance with a mocked db_url
        self.movie_service = MovieService()
    
    def test_get_movie_by_title(self):
        # Call the method with the expected title
        result = self.movie_service.query_by_title(search_string = "Star Wars: Episode I - The Phantom Menace", exact_match = True)
        
        # Assertions to check if the result is as expected
        self.assertIsNotNone(result)
        self.assertEqual(result.iloc[0].movie_name, "Star Wars: Episode I - The Phantom Menace")
        self.assertEqual(result.iloc[0].movie_release_date, datetime.date(1999, 5, 19))

# Run the tests in the notebook
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.364s

OK


## Wildcard search title


In [6]:
import unittest
import datetime



# given: a movie title

# when: we query the database for the movie title

# then: it should return the data for this movie

# Test case for the MovieService class
class TestMovieService(unittest.TestCase):
    def setUp(self):
        # Create a MovieService instance with a mocked db_url
        self.movie_service = MovieService()
    
    def test_get_movie_by_title(self):
        # Call the method with the expected title
        result_df = self.movie_service.query_by_title(search_string = "Kung Fu Panda")
        
        # Assertions to check if the result is as expected
        self.assertIsNotNone(result_df)
        expected_movie_names = ["Kung Fu Panda", "Kung Fu Panda: Secrets of the Furious Five", \
        "Kung Fu Panda 2", "Kung Fu Panda Holiday", "Kung Fu Panda: Secrets of the Masters", \
        "Kung Fu Panda 3", "Kung Fu Panda - The Midnight Stranger Vol.4", "Kung Fu Panda: Legends of Awesomeness 1 : The Scorpion Sting", \
        "Kung Fu Panda: Legends of Awesomeness (Good Croc, Bad Croc)", "Kung Fu Panda: Secrets of the Scroll", "Kung Fu Panda:  The Ultimate Secrets Collection", \
        "Kung Fu Panda: Unstoppable Awesomeness", "Kung Fu Panda: The Emperor's Quest"]

        actual_movie_names = result_df['movie_name']

        # Convert to set for easier comparison
        expected_set = set(expected_movie_names)
        actual_set = set(actual_movie_names)

        # Assert that all expected movies are in the actual DataFrame
        assert expected_set.issubset(actual_set), f"Missing movies in DataFrame: {expected_set - actual_set}"

        # Assert that the actual DataFrame contains no extra movies other than those expected
        assert actual_set.issubset(expected_set), f"Unexpected movies in DataFrame: {actual_set - expected_set}"


# Run the tests in the notebook
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.606s

OK


In [9]:
import unittest
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from db.movie_app_db import MovieDetails

# Create the engine
engine = create_engine(database_path)  # Replace with your actual database URI

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

class TestMovieService(unittest.TestCase):
    def setUp(self):
        # Create a MovieService instance with a mocked db_url
        self.movie_service = MovieService()
    
    def test_get_movie_by_genre(self):
        # Call the method with the expected title
        result_df = self.movie_service.query_by_genre(genre_name = "Action")
        
        # Assertions to check if the result is as expected
        self.assertIsNotNone(result_df)
        # Assuming MovieDetails is a mapped class from SQLAlchemy and result_df is your dataframe with a 'movie_id' column

        # Convert the movie_id column of result_df to a list
        movie_ids = result_df['movie_id'].tolist()

        # Query MovieDetails for all movies with movie_id in movie_ids
        movies = session.query(MovieDetails).filter(MovieDetails.movie_id.in_(movie_ids)).all()

        # Convert the result to a pandas DataFrame
        movies_df = pd.DataFrame([movie.__dict__ for movie in movies])

        # Remove the SQLAlchemy internal attributes (like _sa_instance_state) if needed
        movies_df = movies_df.drop(columns=['_sa_instance_state'], errors='ignore')
        # Assert that every row in the 'genres' column contains the value 'Action'
        assert movies_df['genres'].apply(lambda x: 'Action' in x).all(), "Not all rows contain the genre 'Action'"

# Run the tests in the notebook
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 24.268s

OK


In [4]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from db.movie_app_db import MovieDetails

# Create the engine
engine = create_engine(database_path)  # Replace with your actual database URI

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Query the view for movies with "Kung Fu Panda" in their name
search_string = "Kung Fu Panda"
# movies = session.query(MovieDetails).filter(MovieDetails.movie_name.like(f"%{search_string}%")).all()
# Example: Query the view for a specific movie by its ID
movie_id = 14  # Replace with the actual movie ID you want to search for
movie = session.query(MovieDetails).filter(MovieDetails.movie_id == movie_id).first()

# Print the matching movies
# for movie in movies:
print(movie)

# Optional: Closing session
session.close()


<MovieDetails(movie_id=14, movie_name=American Beauty, movie_release_date=1999-09-15, credits=Lisa Cloud,Sue Casey,Reshma Gajjar,Lily Houtkin,Heather Joy Sher,John Cho,Wes Bentley,Chelsea Hertford,Kent Faulcon,Matthew Kimbrough,Alison Faulk,Erin Cathryn Strubbe,Elaine Corral Kendall,Amber Smith,Scott Bakula,Chekesha Van Putten,Stephanie Rizzo,Barry Del Sherman,Bruce Cohen,Mona Leah,Carolina Lancaster,Krista Goodsitt,Allison Janney,Tom Miller,Kevin Spacey,Brenda Wehle,Peter Gallagher,Emily Zachary,Chris Cooper,Sam Robards,Nancy Anderson,Thora Birch,Marissa Jaret Winokur,Mena Suvari,Dennis Anderson,Annette Bening,Joel McCrary,Hal Fort Atkinson,David C. Fisher,Ara Celi, genres=Drama, studios=Jinks/Cohen Company,DreamWorks Pictures, keywords=marijuana,closeted homosexual,satire,exercise,coming of age,first time,coming out,love affair,gay,parent child relationship,adultery,estate agent,quitting a job,virgin,suburbia,dark comedy,loneliness,extramarital affair,teenager,cheerleader,midlife cri